Data Wrangling Notebook for VertNet Bats Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [96]:
import pandas as pd
import numpy as np
import multiprocessing
import re
import uuid 

Silencing warnings that are unnecessary

In [97]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Import Vertnet Bats Data

In [98]:
df = pd.read_csv("./../Original_Data/bats_2020-08-11b.csv")

Clean up lifeStage

In [99]:
# Fill in NA
df["lifestage_cor"] = df['lifestage_cor'].fillna("Not Collected")

# Create Filters
adult = df['lifestage_cor']=="Adult"
juvenile = df['lifestage_cor']=="Juvenile"
ns = df['lifestage_cor']=="NS"

# Assign correct terms using filters
df['lifestage_cor'][adult] = "adult"
df['lifestage_cor'][juvenile] = "juvenile"
df['lifestage_cor'][ns] = "Not Collected"

Clean up country column TODO: Need to run with GEOME Validation

In [100]:
#Read GEOME country list
geome_countries = pd.read_csv("./../Mapping Files/geome_country_list.csv")

# Create dictionary after inital GEOME run (create csv file first as a dictionary)
df["country"].unique()

def country_correction(country): 
    """Corrects country column to geome specific country list"""
    if country in geome_countries.values:
        return country
    elif country in country_dictionary.keys():
        return country_dictionary[country]
    else:
        country = "Unknown"
        return country 

#df['country'] = df['country'].apply(country_correction)

Create yearCollected column TODO: Create varbatim filter after validation

In [101]:
# Filling N/As with "Unknown"
df["eventdate"] = df["eventdate"].fillna("Unknown")

# Create yearCollected Column
#df = df.assign(yearCollected = '')

# Creating event date variable
#verbatim_date = df['eventdate']

# Establishing vertnet filter
#vertnet_date_filter = verbatim_date.str.contains("""""")

# Grabbing clean data
#verbatim_date_clean= verbatim_date[vertnet_date_filter==False]


# Captures year within string
def year_search(year):
    """Search string for 4 digit number and pass to correct function"""
    if (re.search(r'\d{4}$', year)):
        return year_cleaner_front(year)
    elif (re.search(r'^\d{4}', year)):
        return year_cleaner_back(year)

def year_cleaner_front(year):
    """Isolate the year at the beginning of the string"""
    cleaned_year = year[len(year)-4:len(year)]
    return cleaned_year

def year_cleaner_back(year):
    """Isolate the year at the end of the string"""
    cleaned_year = year[0:4]
    return cleaned_year

df["yearCollected"] = df["eventdate"].apply(year_search)
df["yearCollected"] = df["yearCollected"].fillna("Unknown")

df = df.assign(verbatimEventDate = df['verbatimeventdate'])

Cleaning reproductivecondition column

In [102]:
df["reproductivecondition"].unique()

# TODO: how to handle things like "T=4.1 x 4.1 mm"

norep_filter = df["reproductivecondition"].str.contains("""non-reproductive|not lactating|no lact.""")
preg_filter = df["reproductivecondition"].str.contains("""parous|1 emb|1 emb X 1 mm""")

df["reproductivecondition"][norep_filter == True] = "non-reproductive"
df["reproductivecondition"][preg_filter == True] = "pregnant"
df["reproductivecondition"][(preg_filter == False) & (norep_filter == False)] = ""

Clean sex column 

In [103]:
# Clean up sex column 
female = df['sex'] == "female"
male = df['sex'] == "male"
df['sex'][(female == False) & (male==False)] = ""

Adding additional required GEOME columns

In [104]:
df = df.assign(samplingProtocol = "Unknown")
df = df.assign(basisOfRecord = "PreservedSpecimen")

Rearrange columns so that template columns are first, followed by measurement values

In [105]:
# Create column list
cols = df.columns.tolist()

# Specify desired columns
cols = ['catalognumber',
        'collectioncode',
        'coordinateuncertaintyinmeters',
        'dynamicproperties',
        'decimallatitude',
        'decimallongitude',
        'verbatimlongitude',
        'verbatimlatitude',
        'verbatimelevation',
        'institutioncode',
        'verbatimEventDate',
        'verbatimelevation',
        'verbatimlocality',
        'maximumelevationinmeters',
        'minimumelevationinmeters',
        'reproductivecondition',
        'locality',
        'fieldnotes',
        'binomial',
        'samplingProtocol',
        'occurrenceid',
        'occurrenceremarks',
        'country',
        'sex',
        'lifestage_cor',
        'basisOfRecord',
        'yearCollected',
        'body_mass.1.value',
        'ear_length.1.value',
        'hind_foot_length.1.value',
        'tail_length.1.value',
        'total_length.1.value',
        'body_mass.1.units_inferred',
        'ear_length.1.units_inferred',
        'hind_foot_length.1.units_inferred',
        'tail_length.1.units_inferred',
        'total_length.1.units_inferred',
        'body_mass.1.estimated_value',
        'ear_length.1.estimated_value',
        'hind_foot_length.1.estimated_value',
        'tail_length.1.estimated_value',
        'total_length.1.estimated_value']

# Subset dataframe
df = df[cols]

Matching template and column terms

In [106]:
# Renaming columns 
df = df.rename(columns = {'catalognumber':'catalogNumber',
                          'collectioncode': 'collectionCode',
                          'institutioncode':'institutionCode',
                          'coordinateuncertaintyinmeters':'coordinateUncertaintyInMeters',
                          'fieldnotes':'eventRemarks',
                          'decimallongitude': 'decimalLongitude',
                          'decimallatitude':'decimalLatitude',
                          'occurrenceid':'occurrenceID',
                          'occurrenceremarks':'occurrenceRemarks',
                          'verbatimelevation':'verbatimElevation',
                          'binomial':'scientificName',
                          'reproductivecondition':'reproductiveCondition',
                          'maximumelevationinmeters':'maximumElevationInMeters',
                          'dynamicproperties':'dynamicProperties',
                          'minimumelevationinmeters':'minimumElevationInMeters',
                          'verbatimlocality':'verbatimLocality',
                          'verbatimlongitude':'verbatimLongitude',
                          'verbatimlatitude':'verbatimLatitude',
                          'lifestage_cor':'lifeStage'})

Create materialSampleID which is a UUID for each measurement

In [107]:
df = df.assign(materialSampleID = '')
df['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(df.index))]

Create eventID and populate it with materialSampleID

In [108]:
df = df.assign(eventID = df["materialSampleID"])

Add required GEOME column locality after reassigning locality to verbatimLocality

In [109]:
df = df.assign(locality="Unknown")

Creates a unique measurementMethod column for each desired trait

In [110]:
# List of desired traits
trait_name_list = ["body_mass.1","ear_length.1","hind_foot_length.1",
                   "tail_length.1","total_length.1"]

method_list = ["measurementMethod_" + x for x in trait_name_list]
df = df.join(pd.DataFrame(index = df.index, columns= method_list))

def trait_method(trait):
    """
    Adds measurementMethod information based off of "True" values in inferred value
    and estimated value columns
    """
    
    column = "measurementMethod_" + trait
    
    inferred_column = trait + ".units_inferred"
    estimated_column = trait + ".estimated_value"
    
    inferred_filter = df[inferred_column].astype(str).str.contains("TRUE|True|true")
    estimated_filter = df[estimated_column].astype(str).str.contains("TRUE|True|true")
    
    df[column][inferred_filter] = "Extracted with Traiter ; inferred value"
    df[column][estimated_filter] = "Extracted with Traiter ; estimated value"
    df[column][estimated_filter & inferred_filter] = "Extracted with Traiter ; estimated value; inferred value"
    
[trait_method(x) for x in trait_name_list]

df = df.drop(columns = ['body_mass.1.units_inferred',
                        'ear_length.1.units_inferred',
                        'hind_foot_length.1.units_inferred',
                        'tail_length.1.units_inferred',
                        'total_length.1.units_inferred',
                        'body_mass.1.estimated_value',
                        'ear_length.1.estimated_value',
                        'hind_foot_length.1.estimated_value',
                        'tail_length.1.estimated_value',
                        'total_length.1.estimated_value'])

Creating long version, first specifiying keep variables, then naming type and value

In [114]:
melt_cols = ['catalogNumber', 'collectionCode', 'decimalLatitude','decimalLongitude',
            'verbatimElevation','yearCollected','basisOfRecord','verbatimEventDate',
            'institutionCode','lifeStage','verbatimLocality','locality', 'eventRemarks',
            'samplingProtocol','country','sex','scientificName','dynamicProperties',
            'materialSampleID','eventID','maximumElevationInMeters', 'verbatimLongitude',
            'minimumElevationInMeters','coordinateUncertaintyInMeters','verbatimLatitude',
            'occurrenceID','occurrenceRemarks','reproductiveCondition']

melt_cols = melt_cols + method_list

df_long = pd.melt(df,id_vars = melt_cols, var_name = 'measurementType', value_name = 'measurementValue')

Pull corresponding column value in measurement_method etc and append it to offical measurementMethod

In [ ]:
df_long = df_long.assign(measurementMethod = "")

def method_add(trait,ind):
    if trait == "body_mass.1.value":
        return df_long["measurementMethod_body_mass.1"][ind]
    elif trait == "ear_length.1.value":
        return df_long["measurementMethod_ear_length.1"][ind]
    elif trait == "hind_foot_length.1.value":
        return df_long["measurementMethod_hind_foot_length.1"][ind]
    elif trait == "tail_length.1.value":
        return df_long["measurementMethod_tail_length.1"][ind]
    elif trait == "total_length.1.value":
        return df_long["measurementMethod_total_length.1"][ind]

df_long['ind'] = np.arange(len(df_long))

df_long['measurementMethod'] = df_long.apply(lambda x: method_add(x.measurementType, x.ind), axis=1)

df_long['measurementMethod'] = df_long['measurementMethod'].fillna("Extracted with Traiter")

df_long = df_long.drop(columns = method_list)

Matching trait and ontology terms

In [ ]:
# Create trait dictionary 
trait_dict = {'body_mass.1.value':'body mass',
              'ear_length.1.value': 'ear length to notch',
              'hind_foot_length.1.value':'pes length',
              'tail_length.1.value':'tail length',
              'total_length.1.value':'body length'}

def trait_rename(trait): 
    """
    Renames trait names with trait dictionary
    """
    if trait in trait_dict.keys():
        return trait_dict[trait]

df_long['measurementType'] = df_long['measurementType'].apply(trait_rename)

Populating measurementUnit column with appropriate measurement units in long version

In [ ]:
# Create measurementUnit column
df_long = df_long.assign(measurementUnit="")

#Create filters
long_body_mass_filter = df_long['measurementType']=="body mass"
long_no_body_filter = df_long['measurementType']!="body mass"

#Assign units using filters
df_long['measurementUnit'][long_body_mass_filter] = "g"
df_long['measurementUnit'][long_no_body_filter] = "mm"

Create diagnosticID which is a unique number for each measurement

In [ ]:
df_long = df_long.assign(diagnosticID = '')
df_long['diagnosticID'] = np.arange(len(df_long))

If measurement value equals N/A, delete entire row. Drop range values. 

In [ ]:
#Drop N/A
df_long = df_long.dropna(subset=['measurementValue'])

In [11]:
len(df_long)

NameError: name 'df_long' is not defined